In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
filepath = "../cleaned_data.csv"

In [3]:
df = pd.read_csv(filepath)
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.003497,0,0,0.0,5.0,212.0,3.0,144.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
1,0.036724,34,311,0.0,1.0,62.0,1.0,339.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
2,384.518261,15072,0,0.0,48.0,16416.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
3,0.270332,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
4,0.111429,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47998,16.002137,212,0,0.0,2.0,268.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
47999,32.008281,318,0,0.0,3.0,402.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
48000,47.719591,385,0,0.0,4.0,497.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
48001,32.008033,318,0,0.0,3.0,402.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0


In [4]:
df['label'].value_counts()

Benign                       26001
PartOfAHorizontalPortScan    12369
C&C                           5618
Attack                        3814
Okiru                          163
DDoS                            36
FileDownload                     2
Name: label, dtype: int64

In [5]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']].values

In [6]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df['label']

0        1
1        1
2        1
3        1
4        1
        ..
47998    1
47999    1
48000    1
48001    1
48002    1
Name: label, Length: 48003, dtype: int32

In [7]:
df['label'].unique()

array([1, 6, 2, 0, 5, 3, 4])

In [8]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']

In [9]:
Y = pd.get_dummies(df['label']).values

In [10]:
X.shape

(48003, 24)

In [11]:
scaler = MinMaxScaler()

In [12]:
scaler.fit(X)

MinMaxScaler()

In [13]:
normalized_x = scaler.transform(X)
normalized_x

array([[4.43554718e-08, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.65802215e-07, 1.94861339e-08, 9.24175004e-07, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.87717726e-03, 8.63808855e-06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.05268794e-04, 2.20651811e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.05985532e-04, 1.82252664e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.02965933e-04, 1.21501776e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [14]:
normalized_x.shape

(48003, 24)

In [15]:
scaler.fit(Y)

MinMaxScaler()

In [16]:
normalized_y = scaler.transform(Y)
normalized_y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [18]:
model = Sequential()

In [19]:
Y.shape

(48003, 7)

In [20]:
X

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,proto_icmp,proto_tcp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.003497,0,0,0.0,5.0,212.0,3.0,144.0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0.036724,34,311,0.0,1.0,62.0,1.0,339.0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,384.518261,15072,0,0.0,48.0,16416.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0.270332,48,48,0.0,1.0,76.0,1.0,76.0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.111429,48,48,0.0,1.0,76.0,1.0,76.0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47998,16.002137,212,0,0.0,2.0,268.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
47999,32.008281,318,0,0.0,3.0,402.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
48000,47.719591,385,0,0.0,4.0,497.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
48001,32.008033,318,0,0.0,3.0,402.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [21]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.003497,0,0,0.0,5.0,212.0,3.0,144.0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0.036724,34,311,0.0,1.0,62.0,1.0,339.0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,384.518261,15072,0,0.0,48.0,16416.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,0.270332,48,48,0.0,1.0,76.0,1.0,76.0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0.111429,48,48,0.0,1.0,76.0,1.0,76.0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47998,16.002137,212,0,0.0,2.0,268.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
47999,32.008281,318,0,0.0,3.0,402.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
48000,47.719591,385,0,0.0,4.0,497.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
48001,32.008033,318,0,0.0,3.0,402.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [22]:
scaler = MinMaxScaler()

In [23]:
scaler.fit(X)

MinMaxScaler()

In [24]:
normalized_x = scaler.transform(X)

In [25]:
normalized_x

array([[4.43554718e-08, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.65802215e-07, 1.94861339e-08, 9.24175004e-07, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.87717726e-03, 8.63808855e-06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.05268794e-04, 2.20651811e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.05985532e-04, 1.82252664e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.02965933e-04, 1.21501776e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [26]:
normalized_x.shape

(48003, 24)

In [27]:
scaler.fit(Y)

MinMaxScaler()

In [28]:
normalized_y = scaler.transform(Y)

In [29]:
normalized_y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [31]:
model = Sequential()

In [32]:
X_train.shape

(38402, 24)

In [33]:
# define the features and target
features = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts',
            'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ',
            'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0',
            'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
target = 'label'


# encode the target variable
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# split the data into train and test sets
X = df[features]
Y = df[target]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# scale the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# convert the target variable to categorical
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)


In [34]:
model = Sequential()
model.add(Dense(2000, activation='relu', input_dim=len(features)))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              50000     
                                                                 
 dense_1 (Dense)             (None, 1500)              3001500   
                                                                 
 dropout (Dropout)           (None, 1500)              0         
                                                                 
 dense_2 (Dense)             (None, 800)               1200800   
                                                                 
 dropout_1 (Dropout)         (None, 800)               0         
                                                                 
 dense_3 (Dense)             (None, 400)               320400    
                                                                 
 dropout_2 (Dropout)         (None, 400)              

In [36]:
# Train the model
start = time.time()

model.fit(X_train, Y_train, epochs=10, batch_size=256, validation_data=(X_test, Y_test), verbose=1)

print()
y_pred = model.predict(X_test)

end = time.time()
print('Time Cost:',end - start, 'seconds')

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
print('Accuracy:', test_accuracy)

Epoch 1/10
151/151 [==============================] - 13s 79ms/step - loss: 0.8718 - accuracy: 0.6030 - val_loss: 0.8313 - val_accuracy: 0.6287
Epoch 2/10
151/151 [==============================] - 14s 91ms/step - loss: 0.8222 - accuracy: 0.6167 - val_loss: 0.8015 - val_accuracy: 0.6291
Epoch 3/10
151/151 [==============================] - 13s 85ms/step - loss: 0.8129 - accuracy: 0.6252 - val_loss: 0.8056 - val_accuracy: 0.6288
Epoch 4/10
151/151 [==============================] - 13s 86ms/step - loss: 0.8095 - accuracy: 0.6260 - val_loss: 0.8116 - val_accuracy: 0.6306
Epoch 5/10
151/151 [==============================] - 13s 85ms/step - loss: 0.8097 - accuracy: 0.6274 - val_loss: 0.8020 - val_accuracy: 0.6305
Epoch 6/10
151/151 [==============================] - 13s 85ms/step - loss: 0.8083 - accuracy: 0.6272 - val_loss: 0.8052 - val_accuracy: 0.6305
Epoch 7/10
151/151 [==============================] - 13s 85ms/step - loss: 0.8055 - accuracy: 0.6274 - val_loss: 0.8008 - val_accuracy:

In [37]:
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()

y_pred = classifier_tree.fit(X_train, Y_train).predict(X_test)
print("Classification Report:")
print(classification_report(Y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       756
           1       0.84      0.96      0.90      5253
           2       0.77      0.29      0.42      1111
           3       0.60      0.67      0.63         9
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00        31
           6       0.96      0.95      0.96      2441

   micro avg       0.88      0.88      0.88      9601
   macro avg       0.60      0.55      0.56      9601
weighted avg       0.87      0.88      0.86      9601
 samples avg       0.88      0.88      0.88      9601



c:\Users\JAYDIP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\JAYDIP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\JAYDIP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi